In [49]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [50]:
SEVERITY_COLORS = {
    "Mild Acne": (0, 255, 0),        # Green
    "Moderate Acne": (0, 255, 255),  # Yellow
    "Severe Acne": (0, 0, 255)       # Red
}

In [51]:
# Haar Cascade for face detection
face_cap = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [52]:
# Transfer learning setup for acne detection
IMG_SIZE = (224, 224)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(len(['No Acne', 'Mild Acne', 'Moderate Acne', 'Severe Acne']), activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=output)

In [53]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

In [54]:
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

In [55]:
# Class labels and regions of interest (ROIs)
CLASSES = ['No Acne', 'Mild Acne', 'Moderate Acne', 'Severe Acne']
REGIONS = {
    "forehead": (50, 50, 300, 100),  # x, y, width, height
    "chin": (150, 400, 300, 100),
    "cheek_left": (30, 150, 150, 150),
    "cheek_right": (330, 150, 150, 150),
    "nose": (200, 200, 100, 150)
}

In [56]:
def preprocess_image(image):
    """Preprocess the image for model prediction."""
    image = cv2.resize(image, IMG_SIZE)
    image = image / 255.0  # Normalize to [0, 1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

In [57]:

# def draw_regions(face_roi, regions):
#     """Draw rectangles on regions of interest within the detected face."""
#     for region, (rx, ry, rw, rh) in regions.items():
#         # Check if the ROI is within the bounds of the face image
#         if ry + rh > face_roi.shape[0] or rx + rw > face_roi.shape[1]:
#             print(f"Skipping {region}: Out of bounds.")
#             continue
        
#         # Draw rectangle and label
#         cv2.rectangle(face_roi, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 2)
#         cv2.putText(face_roi, region, (rx, ry - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

def draw_regions(frame, face_roi, regions, age):
    for region, (rx, ry, rw, rh) in regions.items():
        if ry + rh > face_roi.shape[0] or rx + rw > face_roi.shape[1]:
            print(f"Skipping {region}: Out of bounds.")
            continue

        roi = face_roi[ry:ry + rh, rx:rx + rw]
        if roi.size == 0:
            print(f"Error: The ROI for region {region} is empty.")
            continue

        # Predict acne severity
        prediction = predict_acne(roi)
        color = SEVERITY_COLORS.get(prediction, (255, 255, 255))

        # Draw bounding box
        cv2.rectangle(face_roi, (rx, ry), (rx + rw, ry + rh), color, 2)

        # Draw label background
        label = f"{region}: {prediction}"
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(face_roi, (rx, ry - 20), (rx + text_width, ry), (0, 0, 0), -1)

        # Draw label text
        cv2.putText(face_roi, label, (rx, ry - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Show recommendation (optional)
        if region == "forehead":
            recommendation = get_recommendations(age, prediction)
            cv2.putText(frame, recommendation, (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)


In [58]:
def predict_acne(image):
    """Predict acne severity using the loaded model."""
    if image is None:
        raise ValueError("Image is empty. Check if the image is loaded correctly.")
    preprocessed = preprocess_image(image)
    predictions = model.predict(preprocessed)
    return CLASSES[np.argmax(predictions)]

In [59]:
def get_recommendations(age, acne_type):
    """Provide acne treatment recommendations based on age and acne severity."""
    recommendations = {
        "No Acne": "Maintain a healthy skincare routine.",
        "Mild Acne": "Use gentle cleansers and moisturizers. Consider over-the-counter acne products with salicylic acid.",
        "Moderate Acne": "Use products containing benzoyl peroxide or consult a dermatologist for topical treatments.",
        "Severe Acne": "Consult a dermatologist for prescription medications or therapies."
    }
    if age < 18:
        age_specific = "Avoid harsh treatments. Focus on gentle skincare."
    elif 18 <= age <= 25:
        age_specific = "Balance oil control and hydration. Consider professional treatments if needed."
    else:
        age_specific = "Use anti-aging products along with acne treatments to maintain skin health."

    return f"{recommendations[acne_type]} {age_specific}"

In [60]:
def get_relative_regions(face_width, face_height):
    return {
        "forehead": (int(0.2 * face_width), int(0.05 * face_height), int(0.6 * face_width), int(0.2 * face_height)),
        "chin": (int(0.2 * face_width), int(0.75 * face_height), int(0.6 * face_width), int(0.2 * face_height)),
        "cheek_left": (int(0.05 * face_width), int(0.35 * face_height), int(0.3 * face_width), int(0.3 * face_height)),
        "cheek_right": (int(0.65 * face_width), int(0.35 * face_height), int(0.3 * face_width), int(0.3 * face_height)),
        "nose": (int(0.4 * face_width), int(0.35 * face_height), int(0.2 * face_width), int(0.3 * face_height)),
    }


In [61]:
def main():
    # Access the webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not access the camera.")
        return

    age = int(input("Enter your age: "))  # Prompt user to input age

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cap.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(100, 100),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        for (x, y, w, h) in faces:
            # Draw rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Extract the face region for acne detection
            face_roi = frame[y:y + h, x:x + w]
            face_h, face_w = face_roi.shape[:2]

            # Get dynamic regions based on face size
            REGIONS = get_relative_regions(face_w, face_h)

            # Draw ROIs for specific face regions
            # draw_regions(face_roi, REGIONS)
            draw_regions(frame, face_roi, REGIONS, age)

            # Predict acne severity for each region
            for region, (rx, ry, rw, rh) in REGIONS.items():
                # Skip out-of-bounds regions
                if ry + rh > face_h or rx + rw > face_w:
                    print(f"Skipping {region}: Out of bounds.")
                    continue

                roi = face_roi[ry:ry + rh, rx:rx + rw]

                # Check if the region is empty
                if roi.size == 0:
                    print(f"Error: The ROI for region {region} is empty.")
                    continue

                # Perform the prediction
                prediction = predict_acne(roi)
                cv2.putText(face_roi, f"{region}: {prediction}", (rx, ry + rh + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                # Add recommendations for acne severity (for forehead only)
                if region == "forehead":
                    recommendations = get_recommendations(age, prediction)
                    cv2.putText(frame, recommendations, (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Display the frame
        cv2.imshow("Face and Acne Detection", frame)

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


In [62]:
if __name__ == "__main__":
    main()

Enter your age:  21


1/1 [==============================] - 0s 144ms/step
